In [2]:

import numpy as np
from ase.neighborlist import NeighborList, natural_cutoffs

class AtomFeatures:
    def __init__(self, atoms, natural_cutoff_factor=1):
        self.atoms = atoms
        self.natural_cutoff_factor = natural_cutoff_factor
        self.neighbor_list = self.create_neighbor_list()
    
    def get_atom_index(self, interest):
        """
        interest: str or int
            If str, the symbol of the atom of interest.
            If int, the index of the atom of interest.
        """
        if isinstance(interest, str):
            atom_index = next((i for i, atom in enumerate(self.atoms) if atom.symbol == interest), None)
            if atom_index is None:
                raise ValueError(f"No atom with symbol {interest} found.")
        elif isinstance(interest, (int, np.integer)):
            atom_index = interest
        else:
            raise ValueError("Interest must be a string (atom symbol) or an integer (atom index).")
        return atom_index
    
    def create_neighbor_list(self):
        """
        Create a NeighborList object for the atoms object.
            natural_cutoff_factor: float
        """
        cutoffs = natural_cutoffs(self.atoms)
        nl = NeighborList([c * self.natural_cutoff_factor for c in cutoffs], self_interaction=False, bothways=True)
        nl.update(self.atoms)
        return nl
    
    def filter_neighbors(self, atom_index, avoid):
        """
        Filter the neighboring atoms of the atom of interest.
            nl: NeighborList object
            atom_index: int
            atoms: list of ase.Atom objects
            avoid: list of str
                List of atom symbols to avoid. Useful for filtering adsorbate atoms.
        """
        indices, _ = self.neighbor_list.get_neighbors(atom_index)
        filtered_indices = [i for i in indices if self.atoms[i].symbol not in avoid]
        neighboring_atoms = [self.atoms[i].symbol for i in filtered_indices]
        return neighboring_atoms, filtered_indices
    
    def find_neighboring_atoms(self, interest, avoid=[]):
        atom_index = self.get_atom_index(interest)
        neighboring_atoms, filtered_indices = self.filter_neighbors(atom_index, avoid)
        return neighboring_atoms, filtered_indices

    def get_surface_atoms(self, cutoff_cn=12):
        """ 
        Identify surface atoms based on coordination number. 
            natural_cutoff_factor: float - factor to multiply the natural cutoffs by
            cutoff_cn: int - coordination number cutoff for surface atoms
        """
        surface_atoms = []
        for i, atom in enumerate(self.atoms):
            if len(self.neighbor_list.get_neighbors(i)[0]) < cutoff_cn:
                surface_atoms.append(i)
        return surface_atoms

class FeatureCreator:
    def __init__(self, df):
        self.df = df

    def binding_atoms_per_type(self, atoms, ads, metaltype):
        """
        Count the number of specified metal atoms bonded to the adsorbate atom.
            ads: str or int - symbol or index of the adsorbate atom
            metaltype: str - symbol of the metal atom
        """
        af = AtomFeatures(atoms)
        binding_metals, _ = af.find_neighboring_atoms(interest=ads, avoid=[])
        count = len([m for m in binding_metals if m == metaltype])
        return count
    
    def neighbor_count_per_type(self, atoms, ads, metal):
        """
        Count the number of specified metal atoms neighboring the adsorbate atom's neighbors.
            ads: str or int - symbol or index of the adsorbate atom
            metal: str - symbol of the metal atom
        """
        af = AtomFeatures(atoms)
        _, bm_indices = af.find_neighboring_atoms(interest=ads, avoid=[])
        neigh_indexes = []
        metal_symb = []
        for i in bm_indices:
            mneigh, mn_indices = af.find_neighboring_atoms(interest=i, avoid=[ads])
            for j, index in enumerate(mn_indices):
                if index not in neigh_indexes and index not in bm_indices:
                    neigh_indexes.append(index)
                    metal_symb.append(mneigh[j])
        count = len([m for m in metal_symb if m == metal])
        return count

    def add_bonding_features(self, metals=['Ag', 'Au', 'Cu', 'Pd', 'Pt'], ads='N'):
        """
        Add bonding features to the dataframe for specified metals.
        """
        for metal in metals:
            self.df[f'bonding_{metal}'] = self.df.apply(lambda x: self.binding_atoms_per_type(x.Atoms, ads=ads, metaltype=metal), axis=1)

    def add_neigh_features(self, metals=['Ag', 'Au', 'Cu', 'Pd', 'Pt'], ads='N'):
        """
        Add neighboring features to the dataframe for specified metals.
        """
        for metal in metals:
            self.df[f'neigh_{metal}'] = self.df.apply(lambda x: self.neighbor_count_per_type(x.Atoms, ads=ads, metal=metal), axis=1)

In [3]:
from asetools.analysis import check_outcar_convergence
import glob
import pandas as pd
from ase.io import read

mainfolder = '/Users/juanito/Library/CloudStorage/OneDrive-ASTAR/work/computers/HEA/NewCalc_JA/99_N_ads'

# list directories inside mainfolder
dirs = glob.glob(mainfolder + '/*/')

df = pd.DataFrame()
for d in dirs:
    calcfolders = glob.glob(d + '*/')
    for calc in calcfolders:
        try:
            outcar = calc + 'OUTCAR'
            convergence, vasp = check_outcar_convergence(outcar, verbose=False)
        except:
            convergence = False
            vasp = 'Error'
        #print(d.split('/')[-2], calc.split('/')[-2], convergence, vasp)

        if convergence:
            outcar = calc + 'OUTCAR'
            atoms = read(outcar, format='vasp-out', index=-1)
            # append to dataframe
            _df = pd.DataFrame.from_dict({'NP': d.split('/')[-2],
                                    'AdsConfig': calc.split('/')[-2], 
                                    'Convergence': convergence,
                                    'Energy': atoms.get_potential_energy(),
                                    'Atoms': atoms}, orient='index').T
            df = pd.concat([df, pd.DataFrame(_df, index=[0])], ignore_index=True)

In [5]:
fc = FeatureCreator(df)
fc.add_bonding_features()

In [6]:
fc.df

,NP,AdsConfig,Convergence,Energy,Atoms,bonding_Ag,bonding_Au,bonding_Cu,bonding_Pd,bonding_Pt
0,MS,61,True,-178.925206,"(Atom('Cu', [12.29353, 10.04312, 11.43018], in...",1,0,0,1,1
1,MS,59,True,-178.477362,"(Atom('Cu', [12.29447, 9.99291, 11.41855], ind...",2,0,0,0,1
2,MS,66,True,-178.057637,"(Atom('Cu', [12.26958, 10.02349, 11.4701], ind...",1,1,0,1,0
3,MS,50,True,-179.153085,"(Atom('Cu', [12.2886, 10.00522, 11.44912], ind...",0,0,0,1,2
4,MS,68,True,-178.013614,"(Atom('Cu', [12.27315, 10.03009, 11.47151], in...",1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
335,34152,13,True,-176.85021,"(Atom('Cu', [12.31023, 9.92137, 11.39019], ind...",1,1,1,0,0
336,34152,5,True,-177.409889,"(Atom('Cu', [12.29632, 9.94949, 11.39643], ind...",1,0,2,0,0
337,34152,14,True,-176.41006,"(Atom('Cu', [12.2686, 10.00535, 11.40547], ind...",1,1,1,0,0
338,34152,22,True,-177.944044,"(Atom('Cu', [12.27882, 9.99101, 11.43058], ind...",1,0,1,0,1


In [7]:
fc.add_neigh_features()

In [8]:
fc.df

,NP,AdsConfig,Convergence,Energy,Atoms,bonding_Ag,bonding_Au,bonding_Cu,bonding_Pd,bonding_Pt,neigh_Ag,neigh_Au,neigh_Cu,neigh_Pd,neigh_Pt
0,MS,61,True,-178.925206,"(Atom('Cu', [12.29353, 10.04312, 11.43018], in...",1,0,0,1,1,3,1,2,2,3
1,MS,59,True,-178.477362,"(Atom('Cu', [12.29447, 9.99291, 11.41855], ind...",2,0,0,0,1,0,1,5,2,3
2,MS,66,True,-178.057637,"(Atom('Cu', [12.26958, 10.02349, 11.4701], ind...",1,1,0,1,0,1,1,1,3,5
3,MS,50,True,-179.153085,"(Atom('Cu', [12.2886, 10.00522, 11.44912], ind...",0,0,0,1,2,4,4,2,0,1
4,MS,68,True,-178.013614,"(Atom('Cu', [12.27315, 10.03009, 11.47151], in...",1,1,0,1,0,2,1,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,34152,13,True,-176.85021,"(Atom('Cu', [12.31023, 9.92137, 11.39019], ind...",1,1,1,0,0,0,3,2,2,4
336,34152,5,True,-177.409889,"(Atom('Cu', [12.29632, 9.94949, 11.39643], ind...",1,0,2,0,0,3,3,2,2,1
337,34152,14,True,-176.41006,"(Atom('Cu', [12.2686, 10.00535, 11.40547], ind...",1,1,1,0,0,1,3,3,1,4
338,34152,22,True,-177.944044,"(Atom('Cu', [12.27882, 9.99101, 11.43058], ind...",1,0,1,0,1,2,2,0,3,4
